In [1]:
#set project path from config.yaml
import pathlib
import yaml

#asdfasdf

with open("config.yaml", "r") as f:
   config = yaml.safe_load(f)

project_root = pathlib.Path(config["project"]["root_path"])

print("Current project path: ", project_root)

Current project path:  D:\Data Repositories\Coughvid


In [2]:
#Dataset currenntly has 3 file formats
    #-webm
    #-ogg
    #-wav

#WAV is uncompressed and has better compatibility with librosa and pyaudioanalysis. ogg and wav need ffmpeg installed locally and different libraries which will result in 
    #a lot of unecessary trouble during proccesing. Here we'll convert all the .ogg and .webm files to .wav to make signal analysis and subsequent feature extraction easier.

#current directories
webm_dir = project_root / "audio_webm"
ogg_dir = project_root / "audio_ogg"
wav_dir = project_root / "audio_wav"

In [3]:
#create a new directory to move all the new and old .wav files in 

import pathlib 

wav_fulldset_dir = project_root / "audio_wav_fulldset"

wav_fulldset_dir.mkdir(parents = True, exist_ok=True)

In [4]:
#copy the existing wav files to the new directory 
import os
import shutil
from tqdm import tqdm

#list for tqdm
files = [f for f in os.listdir(wav_dir) if os.path.isfile(os.path.join(wav_dir, f))]

# Loop through files with a progress bar
for filename in tqdm(files, desc="Copying files"):
    src_file = os.path.join(wav_dir, filename)
    dst_file = os.path.join(wav_fulldset_dir, filename)
    shutil.copy2(src_file, dst_file)  # copy2 preserves metadata

print("All files copied successfully.")


Copying files: 100%|██████████| 3309/3309 [01:32<00:00, 35.90it/s]

All files copied successfully.


In [8]:

ffmpeg_path = r"D:/Software/FFMPEG/bin/ffmpeg.exe"  # <-- set your actual path here

In [ ]:
import subprocess


# Make sure the destination directory exists
os.makedirs(wav_fulldset_dir, exist_ok=True)

# Get all .webm files
webm_files = [f for f in os.listdir(webm_dir) if f.lower().endswith(".webm")]

# Convert each .webm to .wav
for webm_file in tqdm(webm_files, desc="Converting WebM to WAV"):
    webm_path = os.path.join(webm_dir, webm_file)
    wav_file = os.path.splitext(webm_file)[0] + ".wav"
    wav_path = os.path.join(wav_fulldset_dir, wav_file)
    
    try:
        subprocess.run([
            ffmpeg_path,
            "-y",               # overwrite if exists
            "-i", webm_path,    # input file
            "-ac", "1",         # convert to mono
            "-ar", "44100",     # set sample rate to 44.1kHz
            wav_path            # output file
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to convert {webm_file}: {e}")

print("All WebM files converted to WAV successfully.")


Converting WebM to WAV:   0%|          | 0/29348 [00:00<?, ?it/s]

Converting WebM to WAV: 100%|██████████| 29348/29348 [28:06<00:00, 17.40it/s] 

All WebM files converted to WAV successfully.


In [9]:
import os
from tqdm import tqdm
import subprocess
# Get all .ogg files (case-insensitive)
ogg_files = [f for f in os.listdir(ogg_dir) if f.lower().endswith(".ogg")]

# Convert each .ogg to .wav
for ogg_file in tqdm(ogg_files, desc="Converting OGG to WAV"):
    ogg_path = os.path.join(ogg_dir, ogg_file)
    wav_file = os.path.splitext(ogg_file)[0] + ".wav"
    wav_path = os.path.join(wav_fulldset_dir, wav_file)
    
    try:
        subprocess.run([
            ffmpeg_path,
            "-y",
            "-i", ogg_path,
            "-ac", "1",
            "-ar", "44100",
            "-c:a", "pcm_s16le",  # force 16-bit PCM WAV
            wav_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to convert {ogg_file}: {e}")


Converting OGG to WAV: 100%|██████████| 1777/1777 [01:27<00:00, 20.29it/s]
